This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [1]:
# First, import the relevant modules
import requests

In [2]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json?api_key=&start_date=2014-02-01&end_date=2014-02-02"
res = requests.get(url)

In [3]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json = res.json()
print(json)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume'], 'start_date': '2014-02-01', 'end_date': '2014-02-02', 'frequency': 'daily', 'data': [], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
url_fse = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=&start_date=2017-01-01&end_date=2017-12-31'
result = requests.get(url_fse)
json = result.json()

#### 2. Convert the returned JSON object into a Python dictionary.

In [6]:
fse_dict = dict(json)

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
print(fse_dict['dataset']['column_names'])
prices = fse_dict['dataset']['data']
print(prices[0])
print(len(prices))

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]
255


In [8]:
high_open_price = 0.0
low_open_price = prices[0][1]

for i in prices:
    if i[1] != None:
        if i[1] > high_open_price:
            high_open_price = i[1]
    
        if i[1] < low_open_price:
            low_open_price = i[1]
        
print(f"The highest opening price for the whole year 2017 is: {high_open_price}")
print(f"The lowest opening price for the whole year 2017 is : {low_open_price}")

The highest opening price for the whole year 2017 is: 53.11
The lowest opening price for the whole year 2017 is : 34.0


#### 4. What was the largest change in any one day (based on High and Low price)?

In [9]:
largest_change = 0
for i in prices:
    diff =  i[2] - i[3]
    if diff > largest_change:
        largest_change = diff
        
print(f" The largest change in any one day is : {round(largest_change, 2)}")

 The largest change in any one day is : 2.81


#### 5. What was the largest change between any two days (based on Closing Price)?

In [10]:
prev_price = prices[0][4]
largest_change_two_day = abs(prices[1][4] - prices[0][4])

for i in range(1, len(prices)):
    current_price = prices[i][4]
    diff_price = abs(prev_price - current_price)
               
    potential_price = current_price - prev_price
    largest_change_two_day = max(largest_change_two_day, potential_price )          
    
    prev_price = current_price
               
print(f" The largest change between any two days is : {round(largest_change_two_day, 2)}")

 The largest change between any two days is : 2.56


#### 6. What was the average daily trading volume during this year?

In [11]:
sum = 0
for i in prices:
    sum += i[6]
avg = sum/len(prices)
print(f"The average daily trading volume during this year is : {round(avg, 2)}")

The average daily trading volume during this year is : 89124.34


#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
def calculateMedian(list):
    data = sorted(list)
    n = len(data)
    if n == 0:
        return None
    if n % 2 == 1:
        return data[n // 2]
    else:
        i = n // 2
        return (data[i - 1] + data[i]) / 2

In [13]:
trading_volume = []
for i in prices:
    trading_volume.append(i[6])

median_price = calculateMedian(trading_volume)
print(f" The median trading volume during this year is : {median_price}")

 The median trading volume during this year is : 76286.0
